This code generates the model SNe for all of the models. Check up on Model 4, I think the mixed map has a normalization error. (map built in 3.4cModelVisualization)

In [2]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits, ascii
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
sys.path.append('./Python')
from nearest import findNearest
from deprojectGalaxy import deproject
from SNeGenerationModels import runModels

In [5]:
#Get Galaxy names, distances, and fits file locations

# Unpack master table to get SNe information
database = Table.read('../Data/2.MUSESampleCat.csv', format = "csv")
# centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))]   

database[0:1]

!HERE
Make sure this is full database, I think we are missing galaxy center coords among other things

Galaxy,Distance,Supernova,Type,Ra,Dec,MUSERes,MUSENatMap,MUSE150pcMap,MUSEHII,IRACfiles,InSample
str7,float64,str11,str12,float64,float64,float64,str71,str70,str50,str40,str4
NGC1087,15.85,SN1995V,II,41.61152777777777,-0.4987861111111111,0.92,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1087-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,../Data/IRAC/ngc1087_irac1_atGauss4.fits,True


In [3]:

#Make lists unique for galaxies (no repeating galaxies or other data)    
def unique(origGalsList, origDistList, imageFileList, errFileList, centerCoords, posAngList, inclList, covFileList, ifrFileList, acoFileList, mixedFileList): 
    gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, acoFiles, mixedFiles = [],[],[],[],[],[],[],[],[],[],[]
    for i in range(len(origGalsList)): 
        if origGalsList[i] not in gals: 
            if origGalsList[i] != "ngc1068" and origGalsList[i] != "ngc1672" and origGalsList[i] != "ngc4579":
                gals.append(origGalsList[i]) 
                dists.append(origDistList[i])
                images.append(imageFileList[i])
                errFiles.append(errFileList[i])
                coords.append(centerCoords[i])
                posAngs.append(posAngList[i])
                if inclList[i] == 90.0:
                    inclList[i] = 80.0
                inclinations.append(inclList[i]) 
                covFiles.append(covFileList[i])
                ifrFiles.append(ifrFileList[i])
                acoFiles.append(acoFileList[i])
                mixedFiles.append(mixedFileList[i])
            else:
                pass
    return(gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, acoFiles, mixedFiles)

gals, dists, images, errFiles, coords, posAngs, inclinations, covFiles, ifrFiles, acoFiles, mixedFiles = unique(database["galaxy"], database["dist"],database["ImageFile150"], database["ErrFile150"], centerCoords, database["orient_posang"], database["orient_incl"], database["PACovFile"], database["IFRResizedMaps"], database["AlphaCOFile"], database["MixedMaps"])
print(gals)


['circinus', 'ngc0253', 'ngc0628', 'ngc1087', 'ngc1097', 'ngc1365', 'ngc1433', 'ngc1511', 'ngc1559', 'ngc1566', 'ngc1637', 'ngc2775', 'ngc2997', 'ngc3239', 'ngc3627', 'ngc4254', 'ngc4303', 'ngc4321', 'ngc4424', 'ngc4457', 'ngc4496a', 'ngc4536', 'ngc4945', 'ngc5128', 'ngc5236', 'ngc5530', 'ngc6744', 'ngc7793']


In [4]:
    
# run models 1-4 and store results
galCol, counter, n55mod1, n55mod2, n55mod3, smMod1, smMod2, smMod3 = [],[],[],[],[],[],[],[]

for i in range(len(gals)):
    
    ifr_hdu = pyfits.open(ifrFiles[i])
    ifr_map = ifr_hdu[0].data
    cov_hdu = pyfits.open(covFiles[i])
    cov_map = cov_hdu[0].data

    wcs     = WCS(cov_hdu[0].header, naxis=2)
    naxis   = wcs._naxis # size of image naxis[0] = x and [1] = y
    grid    = np.indices((naxis[1],naxis[0]))
    ra, dec = wcs.wcs_pix2world(grid[1],grid[0],0) 

    f_cov = cov_map.flatten()
    f_ifr = ifr_map.flatten() 
    f_ra  = ra.flatten()
    f_dec = dec.flatten()

    keep = np.where((f_cov > 0.9) & np.isfinite(f_ifr))
    ifr  = f_ifr[keep]
    star_ra   = f_ra[keep]
    star_dec  = f_dec[keep]
    
    
    raArr1, decArr1, dxArr1, dyArr1, smArr1 = runModels(gals[i], images[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=1)
    raArr2, decArr2, dxArr2, dyArr2, smArr2 = runModels(gals[i], images[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)
    raArr3, decArr3, dxArr3, dyArr3, smArr3 = runModels(gals[i], images[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=3, starLight = ifr, starRa = star_ra, starDec = star_dec)
    #raArr4, decArr4, dxArr4, dyArr4 = runModels(gals[i], mixedFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], modelType=2)

    n55_mod1 = nearestMCMethodSigMol(gals[i], images[i], errFiles[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr1, decArr1)
    n55_mod2 = nearestMCMethodSigMol(gals[i], images[i], errFiles[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr2, decArr2)
    n55_mod3 = nearestMCMethodSigMol(gals[i], images[i], errFiles[i], acoFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr3, decArr3)
    #n55_mod4, n65_mod4 = nearestMCMethod(gals[i], images[i], errFiles[i], massFiles[i], coords[i], posAngs[i], inclinations[i], dists[i], raArr4, decArr4)
    
    if (type(n55_mod1) != float and type(n55_mod1) != np.float64):
        for j in range(len(n55_mod1)):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(n55_mod1[j])
            n55mod2.append(n55_mod2[j])
            n55mod3.append(n55_mod3[j])
            #n55mod4.append(n55_mod4[j])
    else:
        for j in range(100):
            galCol.append(gals[i])
            counter.append(j)
            n55mod1.append(np.float('nan'))
            n55mod2.append(np.float('nan'))
            n55mod3.append(np.float('nan'))
            #n55mod4.append(np.float('nan'))  
            
    smArr1    = np.nan_to_num(np.clip(smArr1,0,np.max(smArr1)))
    smArr2    = np.nan_to_num(np.clip(smArr2,0,np.max(smArr2)))
    smArr3    = np.nan_to_num(np.clip(smArr3,0,np.max(smArr3)))
    
    smMod1.append(smArr1)
    smMod2.append(smArr2)
    smMod3.append(smArr3)
smMod1 = np.hstack(smMod1)
smMod2 = np.hstack(smMod2)
smMod3 = np.hstack(smMod3)
    

# cdfSM_mod1 = np.sort(smMod1)/np.sum(smMod1)
# cdfSM_mod2 = np.sort(smMod2)/np.sum(smMod2) 
# cdfSM_mod3 = np.sort(smMod3)/np.sum(smMod3)
    
results = Table({"Galaxy": galCol,
                "ModelNum": counter,
                "n55Mod1": n55mod1,
                "n55Mod2": n55mod2,
                "n55Mod3": n55mod3,
                "SigMolMod1":smMod1,
                "SigMolMod2":smMod2,
                "SigMolMod3":smMod3})

results.write("../Data/AllModels.csv", format = "csv", overwrite = True)

        
        
    

r_sm: [ 2.68216854e+00  3.42925728e+01  2.81326850e+00  6.00200868e+01
  2.97295001e+00  1.48643202e+00  1.66879995e+01 -2.17987402e+00
  1.65767928e+01  5.68987391e+00 -1.21382792e+00  0.00000000e+00
  9.75758960e+00  1.06367492e+00  0.00000000e+00 -4.75061402e-01
  1.07679247e+01 -1.60421215e+00  1.27019547e+01  3.93890756e+00
  1.00638750e+00  3.27854190e-01  4.43877029e-01  1.13689268e+02
  1.53173270e+01  4.97274862e+02  2.17845507e+00  2.62588066e+00
  0.00000000e+00  7.04261776e+00  4.58430773e+01 -1.00748829e+01
 -5.59985407e-01  1.60478961e+01  1.27031218e-01  4.39503536e+00
 -4.58635750e-01  2.15593456e+01  0.00000000e+00 -9.03934038e-01
  4.39682105e-01  6.20562418e+00  6.11453572e+00  5.13893898e+00
 -1.72192453e+00  5.24283210e+00  2.57115166e+00  2.21904109e+01
  3.03742616e+00  8.51342307e+00  4.23448139e+02  6.20234826e+00
  0.00000000e+00 -2.75720258e+00  4.16274691e+01  4.42247454e+00
 -1.05133029e+00  0.00000000e+00 -1.16863294e+01  8.29356476e+00
  2.06715671e+01  1

done with ngc0253  
done with ngc0253  
done with ngc0253  
r_sm: [ 1.06919146e+01  4.97514966e+00  3.35035226e+00  7.87734867e-01
  6.14101529e+00  4.39748188e+00  5.94328016e+00  1.22865897e+01
  3.30447624e+00  2.57475270e+01  2.07235006e+00  1.06811664e+02
  1.55200255e+01  8.21044776e+00  2.86209410e+01  1.19024942e+01
  1.07911928e+01 -1.62084838e+00  3.08219022e+01  1.39152247e+01
  3.40010080e+00  1.40483377e+00  5.74864716e+00 -1.65390560e+00
  1.80364200e+01 -1.18408694e+01  1.54150011e+00  9.93272906e-02
 -1.21554350e+01  3.17322646e+00  8.97245052e+00  1.55824052e+01
  2.40872760e+01  1.23589620e+01  1.96070503e+01  2.63532700e+00
  2.18518108e+01  9.00962621e+01  7.44399521e+00  1.24103546e+01
  1.29997760e+01  1.58123893e+01  1.17892500e+01  1.28868953e+01
  1.01440780e+01  2.21928196e-01  3.43690307e+00  1.28989377e+01
  3.62588335e+00 -1.99973488e+00  3.63595728e+01  9.43641631e+01
  0.00000000e+00  8.98326803e+00 -1.60053888e+00  3.40855316e+00
  1.66992414e+01  8.3058

done with ngc1087  
done with ngc1087  
done with ngc1087  
r_sm: [  0.          28.80618022   5.98196799  -1.74601523   1.36775602
  47.27654538   3.05684073   1.56610234   3.25967023  10.96124271
  19.18281359   3.28476226   0.           0.91320953   2.9206551
  -1.53460152  -5.81576078   0.76212618   6.56247752  -0.39223012
   2.19599628   5.12639035   0.           1.16439912   0.
   5.83854518   8.93057916   0.           0.47935293   3.48686255
  -0.09507614   2.85801945  -0.50841656   6.31063095 -13.86153295
   1.51461799   2.16827558   6.64176437   2.41983423   3.70750946
   0.           4.87821938  -1.3310147    2.21470976   0.
  20.19389753 -10.40107748   3.96755472   1.48990963  -1.38442699
   8.09721975  36.09130861   0.           6.21020069  -0.28177019
  14.35006953  13.35271712  -3.96684033  -1.81720624   0.
  26.16034434  10.54987717  -2.21010661  12.18674098  15.03605913
   7.37539295  -1.86392965   0.63099922   1.80751724   0.
   8.8787903    1.0962241    1.94424645  78

done with ngc1365  
done with ngc1365  
done with ngc1365  
r_sm: [  7.08843128   0.71743718   5.88558864   6.02373544   1.03706931
   0.           6.88633222   5.81074719   0.           0.
  21.2858555   -4.77361686   1.02177068  -2.66417955   1.2920942
   0.           7.04961532   3.41276544   0.           0.
  -3.00475493   0.30723158   0.          -0.763814     0.52056681
   4.20573647   0.           6.94676421  -1.12551678   1.09059321
   8.4693771   -0.47894322   0.           0.           2.28907908
   0.50197574   3.40416217   0.           0.76322274   2.8455146
   0.           0.           0.           0.           4.95031704
   5.22085007   0.          -1.74317605  -0.63019688  -0.10888137
   1.38065182   4.43172529  10.15273321  11.57214563 -10.10404657
   1.93119156   1.87399351   0.           9.0076784   -0.51516192
   0.           1.66613668   1.63986762  -1.00028097   1.07099027
   3.69462839   0.           2.99733661   3.91233824   4.38961113
  10.24196349   5.38789485  

done with ngc1511  
done with ngc1511  
done with ngc1511  
r_sm: [ 5.62988090e+00  0.00000000e+00  1.13503289e+01  9.00121675e+00
  0.00000000e+00  6.63735284e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  8.26209418e+00
  2.00858603e+00  1.17753937e+02  0.00000000e+00  1.68143965e+01
  0.00000000e+00  1.39988789e+00  1.18328233e+01  6.07617255e+00
 -4.92400420e-01  0.00000000e+00  1.08660727e+01  4.00321500e+00
  3.30805553e+01 -1.07002927e+00  1.24306617e+00  0.00000000e+00
  2.36535858e+01  4.86989433e+01  0.00000000e+00  1.27787239e+01
  5.88136241e+01  6.43498084e-01  0.00000000e+00  0.00000000e+00
  3.56574950e+00 -1.18107524e+00  4.25105185e+00  0.00000000e+00
  3.56203239e-01 -1.81389148e+00  0.00000000e+00 -4.18214347e-01
  0.00000000e+00 -5.80364111e-01  5.27571912e+00  0.00000000e+00
 -1.83449510e+00  0.00000000e+00  0.00000000e+00  2.30133623e+00
  4.64838372e+00  4.04108684e+00  0.00000000e+00  3.49726873e+00
  0.00000000e+00  0.0000

...

r_sm: [             inf -2.09410283e+307              inf              inf
              inf  2.03007713e+307  1.19277997e+307              inf
  1.51650067e+307              inf              inf              inf
              inf              inf              inf  2.73631094e+307
  4.04821768e+306              inf  0.00000000e+000  0.00000000e+000
              inf  1.16743571e+307              inf -1.24293170e+307
              inf              inf -2.90009362e+307              inf
              inf -8.96714105e+306 -2.11259216e+307              inf
              inf  6.76678932e+306              inf  5.73862512e+305
              inf              inf              inf              inf
  1.75427407e+307  2.24760316e+307              inf  2.99587810e+307
  0.00000000e+000  0.00000000e+000              inf              inf
  0.00000000e+000  1.18559189e+307              inf  0.00000000e+000
              inf              inf  1.43766582e+307  0.00000000e+000
              inf  0.0000000

<ipython-input-4-7d795ce2a75e>:49: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  n55mod1.append(np.float('nan'))
<ipython-input-4-7d795ce2a75e>:50: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  n55mod2.append(np.float('nan'))
<ipython-input-4-7d795ce2a75e>:51: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. T

r_sm: [ 1.14121776e+00  6.71120421e+01  0.00000000e+00 -4.45219658e-02
  1.28829549e+01 -4.29485159e+00  0.00000000e+00  6.22788874e+00
  6.99554759e+00  1.80254645e+00  8.81633402e-01 -6.40881778e-01
 -8.96023799e-01  3.98910917e-01  1.38513585e+01  2.28486519e+00
  1.78321649e+01  2.65935288e+01  3.68320045e+00  1.30916109e+02
 -5.10733582e+00  6.80938844e-01  4.64007350e+00  0.00000000e+00
  1.15455506e+00 -9.12382087e-01  8.68774850e+00 -1.18287137e+00
  2.34342174e+00  1.92861372e+01 -2.29702341e+00  7.52009371e-02
  4.71471120e+00  2.14951081e+01  3.12232256e-01  0.00000000e+00
  5.30463331e+01  6.70920837e-01  4.86581057e+00  0.00000000e+00
  4.89890653e-01  0.00000000e+00  1.44605314e-01  9.81559587e-01
  0.00000000e+00  3.19649483e+01  5.69428016e+00  7.64199329e+00
 -8.88794928e+00  3.78933933e+01  1.69538878e+00  0.00000000e+00
  2.98784403e-01  0.00000000e+00  2.24732310e+01  6.23428393e+00
 -5.92293115e+00  0.00000000e+00 -1.55678059e+00  6.43828531e+01
 -3.66332151e-02  0

done with ngc5236  
done with ngc5236  
done with ngc5236  
r_sm: [-1.40722764e+00  5.00056875e+00 -3.13632133e+00  0.00000000e+00
  3.82094405e-02  6.32658523e-01  9.26862972e+00  6.66464919e-01
  2.06381793e-01  5.37290039e+00  3.22194077e+00  3.67562637e+00
 -4.23980937e-01  1.64109889e+00 -4.23491999e-01 -1.56750579e+00
 -4.63385480e+00 -3.31876083e+00 -2.67349343e-01  3.57011299e+00
 -1.23547123e+00  2.45476845e+00  1.00193828e+00  4.12697615e+00
  4.87669661e+00 -5.19875641e+00  0.00000000e+00  1.61413256e+01
  1.53024308e+01  1.31404076e+01  1.05064620e+01  8.00667457e+00
 -1.18802389e+00  8.81445608e+00  6.81906125e+00 -7.66182694e-01
  2.58219665e+00  7.70461646e+00  0.00000000e+00  2.96577597e+00
  6.08096387e+00  2.06853112e+00 -1.24507474e-01  3.91929804e+00
 -3.79718880e-01 -3.18063178e+00  1.16606460e+01  3.75619547e+00
  2.22878634e+00  4.92333632e+00 -1.19345131e+00  1.29428617e+00
  6.11873274e+00  2.54872989e+00  3.96742157e+00  2.58360519e-01
 -3.04147901e+00  8.4847

done with ngc6744  
done with ngc6744  
done with ngc6744  
r_sm: [ 0.         -2.95125469  2.25156819 12.92508974 -0.86084086  0.61232235
  0.          0.          1.83277899  6.38702412 -2.02061077  3.91670389
 13.4800863  -3.51914399  0.99690712  9.97487858  5.68984589  6.15683488
  0.          0.04439564 -0.38264038  0.49783785  4.72389743  4.80635841
  0.44598095  9.1805608  -2.0608585   3.14314626  0.          0.61178692
  4.32396251  3.04936507  2.03539826  0.          0.          4.09237279
 -1.51412624  4.59713438  1.99918899  0.          0.          3.16578515
 -2.67277351  0.          5.02653232  3.09485309  5.12900677 -0.4635283
  0.          0.92150147  5.28193421  1.57568247  5.54403229  2.44088152
  6.76850082 -5.51621757  0.60853748  2.82330724 10.11144348 11.37788746
 14.45682117 -2.3226456   8.42937339  0.          0.         -2.78107473
  2.95090457  4.53386916  0.         -0.1728637   3.50585211  4.66778754
  0.          6.07534709  9.13005305  1.79889708  7.4458039

In [5]:
print(len(galCol))
print(len(counter))
print(len(n55mod1))
print(len(n55mod2))
print(len(n55mod3))
np.hstack(smMod1)
# print(len(smMod2))
# print(len(cdfSM_mod3))

2800
2800
2800
2800
2800


array([ 2.68216854, 34.29257282,  2.8132685 , ...,  8.94955935,
        2.10612066,  0.        ])